In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import scipy.misc
import matplotlib.cm as cm
import random
import scipy.optimize
import itertools
from scipy.special import expit
import sys
import math

In [2]:
file = "ex4data1.mat"
data_file = loadmat(file)
initial_data = data_file["X"]
result = data_file["y"]

# insert column of 1's
initial_data = np.insert(initial_data, 0, 1, axis=1)

In [3]:
def get_image_matrix(array):
    # get image size (we know that it will be a square image)
    image_size = int(math.sqrt(array.shape[0]))
    # take all from the first item (exclude 1's)
    return array[1:].reshape(image_size, image_size).T

In [4]:
data_file = "ex4weights.mat"
weight_file = loadmat(data_file)
# theta_1 => size 25 x 401 
# theta_2 => size 10 x 26
theta_1, theta_2 = weight_file['Theta1'], weight_file['Theta2']

In [8]:
input_layer_size = 400 # amount of pixels in one image (without bias)
hidden_layer_size = 25 # take 25 hidden layers because of the provided thetas
output_layer_size = 10 # number of classes

import sklearn
from sklearn.model_selection import train_test_split
data = pd.DataFrame(np.hstack((result, initial_data)))

pd_train_data = data.sample(frac=0.8)
pd_test_data = data.drop(pd_train_data.index)

# Convert training and testing data from Pandas to NumPy format.
train_data = pd_train_data.values
test_data = pd_test_data.values

# Extract training/test labels and features.
num_training_examples = train_data.shape[0]
x_train = train_data[:num_training_examples, 1:]
y_train = train_data[:num_training_examples, [0]]

x_test = test_data[:, 1:]
y_test = test_data[:, [0]]

In [9]:
def unroll(thetas):
    theta_list = [ theta.flatten() for theta in thetas ]
    combined = list(itertools.chain.from_iterable(theta_list))
    assert len(combined) == (input_layer_size+1)*hidden_layer_size + \
                            (hidden_layer_size+1)*output_layer_size
    return np.array(combined).reshape((len(combined),1))

def reshape(unrolled):
    theta1 = unrolled[:(input_layer_size+1)*hidden_layer_size] \
            .reshape((hidden_layer_size,input_layer_size+1))
    theta2 = unrolled[(input_layer_size+1)*hidden_layer_size:] \
            .reshape((output_layer_size,hidden_layer_size+1))
    
    return [ theta1, theta2 ]

In [10]:
def cost_function(unroll_thetas, x, y, lambd=0.):
    #This is what will accumulate the total cost
    total_cost = 0.
    
    thetas = reshape(unroll_thetas)
    
    m = x.shape[0]
    
    # Loop over the training points (rows in myX, already contain bias unit)
    for index in range(m):
        row = x[index]
                
        # First compute the hypothesis (this is a (10,1) vector
        # of the hypothesis for each possible y-value)
        # propagateForward returns (zs, activations) for each layer
        # so propagateforward[-1][1] means "activation for -1st (last) layer"
        
        # get last layer result (hypothesis)
        # [-1] => backwards indexing, [1] => layer output
        hs = propagate_forward(row,thetas)[-1][1]

        # Construct a 10x1 "y" vector with all zeros and only one "1" entry (hot-one encoding)
        hot_one = np.zeros((10,1))
        hot_one[int(y[index])-1] = 1
        
        # Compute the cost for this point and y-vector
        cost = -hot_one.T.dot(np.log(hs))-(1-hot_one.T).dot(np.log(1-hs))
     
        # Accumulate the total cost
        total_cost += cost
  
    # Normalize the total_cost, cast as float
    total_cost = float(total_cost) / m
    
    # Compute the regularization term
    total_reg = 0.
    for theta in thetas:
        total_reg += np.sum(theta*theta) #element-wise multiplication
    total_reg *= float(lambd)/(2*m)
        
    return total_cost + total_reg     

def propagate_forward(features, thetas):
    input_output = []
    for index in range(len(thetas)):  
        theta = thetas[index]
        
        inputs = theta.dot(features).reshape((theta.shape[0], 1))
        # sigmoid hypothesis
        outputs = expit(inputs)
        input_output.append((inputs, outputs))
        if index == len(thetas)-1:
            return np.array(input_output)
        
        features_bias = np.insert(outputs,0,1) # Add the bias unit
        features = features_bias

In [11]:
thetas = unroll([theta_1, theta_2])

# cost without regularization
cost_function(thetas, x_train, y_train)

0.29179609779189913

In [12]:
# cost with regularization
cost_function(thetas, x_train, y_train, 1.)

0.41286938664386796

In [13]:
def predict(row, thetas, classes):
    output = propagate_forward(row, thetas)
    #-1 means last layer, 1 means "a" instead of "z"
    return classes[np.argmax(output[-1][1])] 

def compute_accuracy(thetas, x, y):
    classes = np.unique(result)
    
    n_correct, n_total = 0, x.shape[0]
    for index in range(n_total):
        if int(predict(x[index], thetas, classes)) == int(y[index]): 
            n_correct += 1
    
    return (100*(float(n_correct)/n_total))

In [14]:
# calculate percentage of correct classifications
compute_accuracy([theta_1, theta_2], x_train, y_train)
# it's a bit higher than the regular logistic regression (was ~95% on training data)

97.575

In [15]:
def sigmoid_gradient(z):
    sigmoid = expit(z)
    # derivative of the sigmoid action function
    return sigmoid * (1 - sigmoid)

In [16]:
def generate_random_weights():
    epsilon_init = 0.10
    
    # randomize weight on input => hidden paths
    theta_1 = np.random.rand(hidden_layer_size, input_layer_size+1) * 2 * epsilon_init - epsilon_init
    # randomize weight on hidden => output paths
    theta_2 = np.random.rand(output_layer_size, hidden_layer_size+1) * 2 * epsilon_init - epsilon_init

    return theta_1, theta_2

In [17]:
def back_propagate(unroll_thetas, x, y, lambd=0.):
    #Note: the Delta matrices should include the bias unit
    #The Delta matrices have the same shape as the theta matrices
    Delta1 = np.zeros((hidden_layer_size,input_layer_size+1))
    Delta2 = np.zeros((output_layer_size,hidden_layer_size+1))

    thetas = reshape(unroll_thetas)
    
    # Loop over the training points (rows in myX, already contain bias unit)
    m = x.shape[0]
    for index in range(m):
        row = x[index]
        # reshape into a vector with 401 rows
        a1 = row.reshape((input_layer_size+1,1))
        # propagateForward returns (zs, activations) for each layer excluding the input layer
        temp = propagate_forward(row,thetas)
        z2 = temp[0][0]
        a2 = temp[0][1]
        z3 = temp[1][0]
        a3 = temp[1][1]
        hot_one = np.zeros((10,1))
        hot_one[int(y[index])-1] = 1
        
        delta3 = a3 - hot_one 
        delta2 = thetas[1].T[1:,:].dot(delta3)*sigmoid_gradient(z2) #remove 0th element
        a2 = np.insert(a2,0,1,axis=0)
        Delta1 += delta2.dot(a1.T) #(25,1)x(1,401) = (25,401) (correct)
        Delta2 += delta3.dot(a2.T) #(10,1)x(1,25) = (10,25) (should be 10,26)
        
    D1 = Delta1/float(m)
    D2 = Delta2/float(m)
    
    #Regularization:
    D1[:,1:] = D1[:,1:] + (float(lambd)/m)*thetas[0][:,1:]
    D2[:,1:] = D2[:,1:] + (float(lambd)/m)*thetas[1][:,1:]
    
    return unroll([D1, D2]).flatten()

In [18]:
# back propagate gradients
D1, D2 = reshape(back_propagate(thetas, x_train, y_train, lambd=0.))

In [19]:
def gradient_check(thetas, ds, x, y, lambd=0.):
    eps = 0.0001
    
    for theta_index in range(len(thetas)):
        layer_weights = thetas[theta_index]
        
        for i in range(2):
            for j in range(2):
                row_index = int(np.random.rand()*layer_weights.shape[0])
                column_index = int(np.random.rand()*layer_weights.shape[1])
                
                temp = thetas[theta_index][row_index][column_index]
                minus_eps = thetas[theta_index][row_index][column_index] - eps
                thetas[theta_index][row_index][column_index] = minus_eps
                cost_low = cost_function(unroll(thetas), x, y, lambd)
                
                thetas[theta_index][row_index][column_index] = temp
                
                plus_eps = thetas[theta_index][row_index][column_index] + eps
                thetas[theta_index][row_index][column_index] = plus_eps
                cost_high = cost_function(unroll(thetas), x, y, lambd)
                
                thetas[theta_index][row_index][column_index] = temp
                approx_gradient = (cost_high - cost_low) / float(2 * eps)
                
                print(f'[{row_index}][{column_index}], check gradient: {approx_gradient:f}, BP gradient: {ds[theta_index][row_index][column_index]:f}')

In [20]:
gradient_check([theta_1, theta_2], [D1, D2], x_train, y_train)

[8][350], check gradient: 0.000062, BP gradient: 0.000062
[8][363], check gradient: -0.000000, BP gradient: -0.000000
[18][112], check gradient: -0.000064, BP gradient: -0.000064
[1][282], check gradient: 0.000000, BP gradient: 0.000000
[5][2], check gradient: 0.000582, BP gradient: 0.000582
[6][10], check gradient: -0.000856, BP gradient: -0.000856
[9][20], check gradient: -0.001090, BP gradient: -0.001090
[7][14], check gradient: 0.001315, BP gradient: 0.001315


In [21]:
# back propagate gradients
D1, D2 = reshape(back_propagate(thetas, x_train, y_train, lambd=0.5))

In [ ]:
gradient_check(np.array([theta_1, theta_2]), [D1, D2], x_train, y_train, 0.5)

[10][32], check gradient: 0.000003, BP gradient: 0.000003
[3][16], check gradient: 0.000000, BP gradient: 0.000000
[22][348], check gradient: 0.000009, BP gradient: 0.000009
[10][279], check gradient: -0.000020, BP gradient: -0.000020
[0][10], check gradient: 0.000706, BP gradient: 0.000706
[1][15], check gradient: -0.001211, BP gradient: -0.001211
[6][12], check gradient: -0.000238, BP gradient: -0.000238
[2][22], check gradient: 0.000023, BP gradient: 0.000023


In [ ]:
def train(x, y, lambd = 0., maxiter = 30, disp = True):
    
    theta_1, theta_2 = generate_random_weights()
    
    thetas = unroll([theta_1, theta_2])

    optimized_thetas = scipy.optimize.fmin_cg(cost_function, x0=thetas, fprime=back_propagate,
                                args=(x, y, lambd), maxiter=maxiter, disp=disp)
    
    return reshape(optimized_thetas)

In [ ]:
learned_thetas = train(x_train, y_train, 0., 50)

In [ ]:
compute_accuracy(learned_thetas, x_train, y_train)

In [ ]:
def display_hidden_layer(hidden_layer_theta):
    images = hidden_layer_theta
    
    fig, axs = plt.subplots(5, 5, figsize=(8, 8))
    fig.subplots_adjust(wspace=0.025, hspace=0.025)
    
    axs = axs.flatten()
    
    for i in range(len(axs)):
        ax = axs[i]
        ax.imshow(get_image_matrix(images[i]), cmap='gray')
        ax.axis('off')

    plt.show()

In [ ]:
display_hidden_layer(learned_thetas[0])

In [ ]:
lambda_values = [0, 0.001, 0.01, 0.1, 1., 10., 30.]
lambda_tests = {}
for i in range(len(lambda_values)):
    lambd = lambda_values[i]
    
    thetas = train(x_train, y_train, lambd, maxiter = 50, disp=False)
    accuracy = compute_accuracy(thetas, x_train, y_train)
    
    lambda_tests[str(lambd)] = {
        'lambda': lambd,
        'thetas': thetas,
        'accuracy': accuracy
    }

In [ ]:
lambdas = []
accuracies = []
lambda_tests.items()
for key, value in lambda_tests.items():
    print(f"lambda: {key}, accuracy: {value['accuracy']}")
    lambdas.append(value['lambda'])
    accuracies.append(value['accuracy'])
    display_hidden_layer(np.array(value["thetas"])[0])

In [ ]:
x_values = np.arange(len(lambdas))

plt.figure(figsize=(10,6))
plt.plot(x_values, accuracies, 'o-')
plt.grid(True)
plt.xlabel("\u03BB", fontsize=25)
plt.ylabel("Accuracy", fontsize=25)
plt.xticks(x_values, lambdas)
plt.show()